Задание 1

Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.


In [19]:
import pandas as pd
ratings = pd.read_csv('ml-latest-small/ratings.csv')
ratings.tail(10)

,userId,movieId,rating,timestamp
99994,671,5952,5.0,1063502716
99995,671,5989,4.0,1064890625
99996,671,5991,4.5,1064245387
99997,671,5995,4.0,1066793014
99998,671,6212,2.5,1065149436
99999,671,6268,2.5,1065579370
100000,671,6269,4.0,1065149201
100001,671,6365,4.0,1070940363
100002,671,6385,2.5,1070979663
100003,671,6565,3.5,1074784724


In [61]:
x = (ratings.groupby('userId').agg({'movieId': 'count', 'timestamp': ['min', 'max']}).
     reset_index())
y = x[x['movieId']['count'] > 100]
y['difference'] = y['timestamp']['max'] - y['timestamp']['min']
y['difference'].mean()

<ipython-input-61-2f3a769d5c1b>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y['difference'] = y['timestamp']['max'] - y['timestamp']['min']


40080507.4496124


Задание 2

Дана статистика услуг перевозок клиентов компании по типам:

rzd - железнодорожные перевозки
auto - автомобильные перевозки
air - воздушные перевозки
client_base - адреса клиентов

Необходимо сформировать две таблицы:
таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
аналогичную таблицу по типам выручки с указанием адреса клиента
Обратите внимание, что в процессе объединения таблиц данные не должны теряться.



In [62]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

,client_id,rzd_revenue
0,111,1093
1,112,2810
2,113,10283
3,114,5774
4,115,981


In [63]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

,client_id,auto_revenue
0,113,57483
1,114,83
2,115,912
3,116,4834
4,117,98


In [64]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

,client_id,air_revenue
0,115,81
1,116,4
2,117,13
3,118,173


In [65]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4


In [76]:
#таблица с адресами:
clients_revenues_address = (client_base.merge(air, on='client_id', how='left').
                    merge(auto, on='client_id', how='left').
                    merge(rzd, on='client_id', how='left'))
clients_revenues_address


,client_id,address,air_revenue,auto_revenue,rzd_revenue
0,111,Комсомольская 4,NaN,NaN,1093.0
1,112,Энтузиастов 8а,NaN,NaN,2810.0
2,113,Левобережная 1а,NaN,57483.0,10283.0
3,114,Мира 14,NaN,83.0,5774.0
4,115,ЗЖБИиДК 1,81.0,912.0,981.0
5,116,Строителей 18,4.0,4834.0,NaN
6,117,Панфиловская 33,13.0,98.0,NaN
7,118,Мастеркова 4,173.0,NaN,NaN


In [78]:
#таблица без адресов:
clients_revenues = clients_revenues_address.iloc[:,[0,2,3,4] ]
clients_revenues

,client_id,air_revenue,auto_revenue,rzd_revenue
0,111,NaN,NaN,1093.0
1,112,NaN,NaN,2810.0
2,113,NaN,57483.0,10283.0
3,114,NaN,83.0,5774.0
4,115,81.0,912.0,981.0
5,116,4.0,4834.0,NaN
6,117,13.0,98.0,NaN
7,118,173.0,NaN,NaN


Задание 3

В задаче сквозной аналитики вам предоставили данные по местоположению пользователей. Т. е. для каждого user_id известна последовательность координат (широта/долгота), когда они требовались приложению для полноценной работы. Как бы вы добавили эти сведения в таблицу визитов и покупок? Для составления ответа можно использовать вопросы:
У каждого пользователя известен набор координат. А для связывания с визитом или фактом покупки скорее всего потребуется одно-два числа. Как их получить?
Наборы координат одного и того же пользователя могут быть значительно удалены друг от друга. Как это отразится на вопросе расчетах пункта 1?
Какие дополнительные признаки можно получить из координат? Ведь это просто числа, которые сами по себе мало что дают.

1) Нам нужно время визита, покупки и время получения координат. Доп вопрос: когда приложению требуются координаты пользователя? Тогда можно понять, как связать время получения координат с другими действиями. 
2) Из координат можно понять географическое распределение потенциальных и реальных покупателей. Можно это в будущем использовать для таргетирования. Если нам нужно, то можно рассчитать скорость перемещения покупателей, то есть предположить транспортное средство - автомобиль, пешком. 